Notebook for practicing with the mujoco environment and API

The following are some of the requirements in addition to the `requirements.txt` file.

```
mujoco-python-viewer==0.1.4
ipykernel
```

In [1]:
import mujoco
import mujoco_viewer

# scene.xml includes the robot model and a simple environment
model = mujoco.MjModel.from_xml_path("../unitree_go1/scene_position.xml")

# Contains the state of the model (time: .time, pos: .qpos, vel: .qvel) 
data = mujoco.MjData(model)
model.key_ctrl[0]


array([ 0. ,  0.8, -1.5,  0. ,  0.8, -1.5,  0. ,  1. , -1.5,  0. ,  1. ,
       -1.5])

In [129]:
# Print out the name of the body accessors
try:
    model.geom()
except KeyError as e:
    print(f"model.geom: {e}")
    
try:
    data.geom()
except KeyError as e:
    print(f"data.geom: {e}")
    
try:
    data.body()
except KeyError as e:
    print(f"data.body: {e}")
    
try:
    model.actuator()
except KeyError as e:
    print(f"model.actuator: {e}")
    
    
trunk_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY.value, "trunk")
data.body(trunk_id)

print(f"actuator control gain: {model.actuator(0).gainprm}")
# print(model.geom(5).friction)


model.geom: "Invalid name ''. Valid names: ['FL', 'FR', 'RL', 'RR', 'floor']"
data.geom: "Invalid name ''. Valid names: ['FL', 'FR', 'RL', 'RR', 'floor']"
data.body: "Invalid name ''. Valid names: ['FL_calf', 'FL_hip', 'FL_thigh', 'FR_calf', 'FR_hip', 'FR_thigh', 'RL_calf', 'RL_hip', 'RL_thigh', 'RR_calf', 'RR_hip', 'RR_thigh', 'trunk', 'world']"
model.actuator: "Invalid name ''. Valid names: ['FL_calf', 'FL_hip', 'FL_thigh', 'FR_calf', 'FR_hip', 'FR_thigh', 'RL_calf', 'RL_hip', 'RL_thigh', 'RR_calf', 'RR_hip', 'RR_thigh']"
actuator control gain: [50.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [130]:
print(f"Simulated time: {data.time} sec")
print(f"Default timestep: {model.opt.timestep} sec")
print(f"Degrees of freedom: {model.nv=}")

# Position has 1 extra dimension since orientation is represented with
# quaternions (4-values) while angular velocity has 3 values.
print(f"{len(data.qpos)=} {data.qpos=}")
print(f"{len(data.qvel)=} {data.qvel=}")
print(f"{len(data.xpos)=} {data.xpos=}")
print(f"{len(data.ctrl)=} {data.ctrl=}")
# print(f"{len(data.site_xpos)=} {data.site_xpos=}")

Simulated time: 0.0 sec
Default timestep: 0.002 sec
Degrees of freedom: model.nv=18
len(data.qpos)=19 data.qpos=array([0.   , 0.   , 0.445, 1.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   ])
len(data.qvel)=18 data.qvel=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])
len(data.xpos)=14 data.xpos=array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])
len(data.ctrl)=12 data.ctrl=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [131]:
import numpy as np
import time
# Setup viewer
viewer = mujoco_viewer.MujocoViewer(model, data)

# Reset the simulation
mujoco.mj_resetDataKeyframe(model, data, 0)

print(f"{len(data.ctrl)=} {data.ctrl=}")
print(f"ctrl/torque range defined in motor ctrlrange in xml: {model.actuator_ctrlrange=}")


_cfrc_ext_feet_indices = [4, 7, 10, 13]  # 4:FR, 7:FL, 10:RR, 13:RL
feet_site = [
    "FR",
    "FL",
    "RR",
    "RL",
]
feet_site_indices = [mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_SITE.value, f)
    for f in feet_site]


print([mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_ACTUATOR.value, i)
    for i in range(len(data.qfrc_actuator))])

print(f"{model.jnt_range=}")

for _ in range(10000):
    if viewer.is_alive:
        mujoco.mj_step(model, data)
        mujoco.mj_rnePostConstraint(model, data)
        
        
        feet_contact_force_mag = np.linalg.norm(data.cfrc_ext[_cfrc_ext_feet_indices])
        curr_contact = feet_contact_force_mag > 0.1
        contacting_feet = data.site_xpos[feet_site_indices] * curr_contact
        
        mujoco.mj_fwdActuation(model, data)
        
        data.ctrl[-1] = -2
        
        print(f"{data.ctrl[-1]=}")
        print(f"{data.qpos[-1]=}")
        
        
        viewer.render()
        time.sleep(0.001)
        
    else:
        break

viewer.close()

len(data.ctrl)=12 data.ctrl=array([ 0. ,  0.8, -1.5,  0. ,  0.8, -1.5,  0. ,  1. , -1.5,  0. ,  1. ,
       -1.5])
ctrl/torque range defined in motor ctrlrange in xml: model.actuator_ctrlrange=array([[-0.863,  0.863],
       [-0.686,  4.501],
       [-2.818, -0.888],
       [-0.863,  0.863],
       [-0.686,  4.501],
       [-2.818, -0.888],
       [-0.863,  0.863],
       [-0.686,  4.501],
       [-2.818, -0.888],
       [-0.863,  0.863],
       [-0.686,  4.501],
       [-2.818, -0.888]])
['FR_hip', 'FR_thigh', 'FR_calf', 'FL_hip', 'FL_thigh', 'FL_calf', 'RR_hip', 'RR_thigh', 'RR_calf', 'RL_hip', 'RL_thigh', 'RL_calf', None, None, None, None, None, None]
model.jnt_range=array([[ 0.   ,  0.   ],
       [-0.863,  0.863],
       [-0.686,  4.501],
       [-2.818, -0.888],
       [-0.863,  0.863],
       [-0.686,  4.501],
       [-2.818, -0.888],
       [-0.863,  0.863],
       [-0.686,  4.501],
       [-2.818, -0.888],
       [-0.863,  0.863],
       [-0.686,  4.501],
       [-2.818, -0.88